In [1]:
import torch
from transformers import AutoModel,AutoConfig,AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments,DataCollatorWithPadding
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import re
# from pygtrans import Translate

In [2]:
# %%time
# # 加载数据
# excel_file = 'd:/基于深度学习的海量文本处理/第1阶段/10w.xlsx'
# data_frame = pd.read_excel(excel_file)

In [3]:
# # 随机抽样（预实验使用）
# data_frame = data_frame.sample(n=30)
# data_frame

In [4]:
# # 停用词预处理
# stop_words = ['您好','你好',':很高兴为您服务','请问有什么可以帮您','client','user',' ']
# sep_words = ['。', '!', '?', ',']
# def ProcessStopWords(text):
#     for word in stop_words:
#         text = text.replace(word,'')
#     text = text.replace(':','。').replace('。','',1) # 删除第一个。
#     # for word in sep_words:
#     #     text = text.replace(word, '[SEP]')
#     return text

# data_frame['转写文本'] = data_frame['转写文本'].map(ProcessStopWords)

In [5]:
# data_frame['转写文本'].iloc[0]

In [6]:
# # 标签预处理
# regex = re.compile(r'^.*?>(.*?)>.*?$')
# def ProcessLabels(text):
#     text = text.replace('>>','>').replace('10019','')
#     text = re.match(regex, text).groups()[0]
#     return text

# data_frame['服务请求'] = data_frame['服务请求'].map(ProcessLabels)

In [7]:
# 预览预处理结果
# data_frame['转写文本'].iloc[:10]

In [8]:
# # 训练复盘并分析数据集后，考虑在前面的处理把 样本数<1000 的剔除，即剔除下列：
# rm_labels = ['临时','其他','商机','资料信息','业务变更问题','投诉','故障']
# for rm_label in rm_labels:
#     data_frame.drop(data_frame[data_frame.服务请求 == rm_label].index, inplace=True)
# data_frame

In [9]:
# # 翻译转写文本
# client = Translate(target='en')
# temp = np.array([])

In [10]:
# # 测试
# pd.DataFrame(np.array([trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[0:2]).tolist())]))

In [11]:
# total_num = data_frame.shape[0]
# batch = 100
# epoch = int(total_num/batch)
# print(f'共{epoch}部分')
# for i in range(epoch):
#     if not i == epoch - 1:
#         print(f'{i}部分开始')
#         translated_res = [trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[i*batch:(i+1)*batch]).tolist())]
#     else:
#         print('最后一轮开始')
#         translated_res = [trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[i*batch:]).tolist())]
    
#     translated_res = np.array(translated_res)
#     temp =  np.concatenate((temp, translated_res), axis=0)
#     print(f'第{i}部分已完成')

In [12]:
# translated_res = temp
# len(translated_res)

In [13]:
# # 加入翻译
# # data_frame = data_frame.drop(columns=['translated_text'])
# data_frame.insert(0, 'translated_text', value=translated_res)

In [14]:
# # 处理翻译出错的符号
# def Finetune_translate(text):
#     text = text.replace('&#39;','\'')
#     return text
    
# data_frame['translated_text'] = data_frame['translated_text'].map(Finetune_translate)
# data_frame[:10]

In [15]:
# prompts = np.array(data_frame['translated_text'])
# choices = np.array(data_frame['服务请求'])

In [16]:
# data_frame['服务请求']

In [17]:
# # 取最大长度
# data_frame.insert(data_frame.shape[1], 'text_len',None)
# data_frame['text_len'] = data_frame['translated_text'].map(len)
# max_length_index = data_frame['text_len'].argmax()
# max_length = data_frame['text_len'].iloc[max_length_index]
# max_length_index, max_length

In [18]:
# # 观察文本长度分布（排除异常值）
# data_frame.boxplot('text_len', grid=False, showfliers=False, color='Black')
# plt.suptitle("")
# plt.xlabel("")
# plt.show()
# # 由图可知，取512足够覆盖正常样本
# max_length = 512

In [19]:
# # 去重choices，并保存原choices对应去重后的位置
# unique_choices = np.unique(choices)
# labels = np.array([np.argwhere(unique_choices==v)[0]  for v in choices])
# unique_choices.shape, labels.shape

In [20]:
# # 加入标签
# data_frame.insert(0, 'label', value=labels)
# data_frame[:10]

In [21]:
# # 取出特征与labels
# df = data_frame[['label', 'translated_text', '服务请求']]
# df[:10]
# # 统计
# df['label'].value_counts(ascending=True).plot.barh()
# plt.title("Frequency of Classes")
# plt.show()

In [22]:
# # 手动处理样本非均衡情况
# df4 = df[df['label'] == 4].sample(n=30000)
# df2 = df[df['label'] == 2]
# df1 = pd.DataFrame(np.repeat(df[df['label'] == 1].values, 2, axis=0), columns=df.columns)
# df0 = pd.DataFrame(np.repeat(df[df['label'] == 0].values, 2, axis=0), columns=df.columns)
# df3 = pd.DataFrame(np.repeat(df[df['label'] == 3].values, 2, axis=0), columns=df.columns)

# df = pd.concat([df0, df1, df2, df3, df4], ignore_index=True)

In [23]:
# # 先排序 label，以便后续充分打乱
# df = df.sort_values(by='服务请求')

In [24]:
save_excel_path = './PreProcess.xlsx'

In [25]:
# df.to_excel(save_excel_path) # 保存翻译结果，方便重复使用
# df

In [26]:
# 用于直接读取预处理完成的结果
df = pd.read_excel(save_excel_path)
df = df.drop(columns='Unnamed: 0')
df.dropna(inplace=True) # drop 空行，防止 tokenize 失败

In [27]:
# # 预实验
# df = df.sample(n=1000)

In [28]:
# 根据经验，人为合并相似分类
def CombineLabel(label):
    # 仅映射 label 属性
    label = 2 if label == 4 else label # 查询与咨询合并
    return label

df['label'] = df['label'].map(CombineLabel)
df = df.sort_values(by='label')

In [29]:
# 预处理结果相关参数
max_length = 512
classic_num = 4

# 预览
print(df.shape[0])
df[:10]

99326


,label,translated_text,服务请求
0,0,"Hey, that is my account. The current network i...",不满
7637,0,Hello. When can you solve the problem of chang...,不满
7638,0,". . Excuse me, a few days ago, I asked you whe...",不满
7639,0,", I'm glad to serve you. Ah, your phone has be...",不满
7640,0,I would like to ask my light cat why the red l...,不满
7641,0,"Hello. gentlemen. Hey, why can’t I access the ...",不满
7642,0,"Hey, this is my mobile phone number. I see tha...",不满
7643,0,"at your service. Well, I would like to ask why...",不满
7644,0,"Ah, you just told me that I wanted to check th...",不满
7636,0,"Uh, hey, I took a look at my streaming phone b...",不满


In [30]:
# 创建数据集
ds = DatasetDict({'train': Dataset.from_pandas(df)})
# ds['train'] = ds['train'].rename_column('转写文本','text')
ds = ds.remove_columns(['服务请求'])

ds['train'] = ds['train'].rename_columns({'translated_text':'text'})
ds = ds['train'].train_test_split(0.2, shuffle=True) # 按 8:2 分割数据集

# ds = ds.shuffle(88) # 再次打乱数据集
ds, ds['train'][0], ds['test'][0]

(DatasetDict({
     train: Dataset({
         features: ['label', 'text', '__index_level_0__'],
         num_rows: 79460
     })
     test: Dataset({
         features: ['label', 'text', '__index_level_0__'],
         num_rows: 19866
     })
 }),
 {'label': 0,
  'text': "Hello. Hello, this is your side. Can you help me lift the Internet speed limit? The Internet speed limit cannot be lifted, sir. If it can't be resolved, can I just wait until next month for it to resolve itself? right. If you want to unblock this network setting, you can only use the accelerated traffic package. Then why does he limit the speed even when I use it? For his side, he will have a speed limit rule, but for your side, if the speed is limited, if you want to speed up, you can only buy that kind of data package. ah. Acceleration package. But I have traffic. There is only one kind. Um. Nothing can be solved. That is, only the traffic package acceleration package can be released by purchasing the traffic package

In [31]:
%%time
model_path = './gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token='[PAD]'
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=max_length)

tokenized_ds = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 19866/19866 [00:04<00:00, 4172.35 examples/s]

CPU times: user 1min 30s, sys: 644 ms, total: 1min 31s
Wall time: 23.9 s


In [32]:
# 模型
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 允许不同长度tensor的存在
model_config = AutoConfig.from_pretrained(model_path)
model_config.num_labels = classic_num
model_config.pad_token_id = 0
model = AutoModelForSequenceClassification.from_config(model_config)

In [33]:
tokenized_ds['train'][0]

{'label': 0,
 'text': "Hello. Hello, this is your side. Can you help me lift the Internet speed limit? The Internet speed limit cannot be lifted, sir. If it can't be resolved, can I just wait until next month for it to resolve itself? right. If you want to unblock this network setting, you can only use the accelerated traffic package. Then why does he limit the speed even when I use it? For his side, he will have a speed limit rule, but for your side, if the speed is limited, if you want to speed up, you can only buy that kind of data package. ah. Acceleration package. But I have traffic. There is only one kind. Um. Nothing can be solved. That is, only the traffic package acceleration package can be released by purchasing the traffic package acceleration package, and nothing else will work. Okay, okay, no, no, no, thank you, bye. I wish you a happy life bye",
 '__index_level_0__': 7875,
 'input_ids': [15496,
  13,
  18435,
  11,
  428,
  318,
  534,
  1735,
  13,
  1680,
  345,
  1037,

In [34]:
# 评估
accuracy = evaluate.load('./evaluate/metrics/accuracy/accuracy.py')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [40]:
# Adam 训练阶段
ds = ds.shuffle(88) # 再次打乱数据集
training_args = TrainingArguments('/hy-tmp/SRP-checkpoint-output',evaluation_strategy='epoch',save_strategy='epoch', learning_rate=5e-7, 
                                    load_best_model_at_end=True, num_train_epochs=16)
trainer = Trainer(model, args=training_args, train_dataset=tokenized_ds['train'], eval_dataset=tokenized_ds['test'], 
                  tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics)

In [41]:
%%time
# 训练
trainer.train(resume_from_checkpoint=True)

Epoch,Training Loss,Validation Loss,Accuracy
9,0.752800,0.768141,0.733011
10,0.742100,0.775771,0.732961
11,0.732300,0.757588,0.735629
12,0.748800,0.758290,0.734270
13,0.752100,0.755656,0.736283
14,0.724200,0.755958,0.736132
15,0.710000,0.769064,0.734823
16,0.715800,0.760770,0.736434


CPU times: user 4h 1min 49s, sys: 1min 9s, total: 4h 2min 59s
Wall time: 4h 1min 57s


TrainOutput(global_step=158928, training_loss=0.36715211970777556, metrics={'train_runtime': 14515.0903, 'train_samples_per_second': 87.589, 'train_steps_per_second': 10.949, 'total_flos': 2.138172313524388e+17, 'train_loss': 0.36715211970777556, 'epoch': 16.0})

In [43]:
# 保存
res_path = '/hy-tmp/SRP_Business_Classification/'
tokenizer.save_pretrained(res_path)
model.save_pretrained(res_path)

In [38]:
# # SGD 训练阶段
# # ds = ds.shuffle(88) # 再次打乱数据集
# optimizer = torch.optim.SGD(model.parameters(), lr=5e-8)
# lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))
# training_args = TrainingArguments('/hy-tmp/SRP-output',evaluation_strategy='epoch',save_strategy='epoch',
#                                 load_best_model_at_end=True, num_train_epochs=5)
# trainer = Trainer(model, args=training_args, train_dataset=tokenized_ds['train'], eval_dataset=tokenized_ds['test'], 
#                   tokenizer=tokenizer, data_collator=data_collator, 
#                   compute_metrics=compute_metrics, optimizers=(optimizer, lr_scheduler))

In [39]:
# %%time
# # 精准训练
# trainer.train(resume_from_checkpoint=True)